# Preset

In [37]:
start_date = "01/02/2019"
end_date = "01/05/2019"
market = "RTM"  # DAM for day ahead market, RTM for 5 minute market
node = "HOLLISTR_1_N101" # the node ID
queryname = 'PRC_INTVL_LMP'  # (PRC_LMP for 1 hour prices) PRC_RTPD_LMP

# Libs

In [4]:
import requests
import pandas as pd
import zipfile
import io
import time
from datetime import datetime, timedelta

# Convert US format date (MM/DD/YYYY) to datetime object

In [30]:
def convert_to_iso(us_date, us_time="00:00"):
    # Convert US format date (MM/DD/YYYY) and time (HH:MM) to datetime object
    date_time_str = f"{us_date} {us_time}"
    date_object = datetime.strptime(date_time_str, "%m/%d/%Y %H:%M")
    
    # Convert datetime object to ISO 8601 format
    iso_date = date_object.strftime("%Y%m%dT%H:%M")

    # Manually append "+0000" offset from UTC
    iso_date += "+0700"
    
    return iso_date

def date_range(start, end):
    start_date = datetime.strptime(start, "%m/%d/%Y")
    end_date = datetime.strptime(end, "%m/%d/%Y")
    
    current_date = start_date
    
    while current_date < end_date:
        next_date = current_date + timedelta(days=30)
        if next_date > end_date:
            next_date = end_date

        yield convert_to_iso(current_date.strftime("%m/%d/%Y")), convert_to_iso(next_date.strftime("%m/%d/%Y"))
        current_date = next_date

# Collecting history for the given tameframe

In [38]:
# Initialize an empty DataFrame to store all data
df_total = pd.DataFrame()
monthes = 0
for start, end in date_range(start_date, end_date):
    print(f"Start: {start}, End: {end}")

    # The base URL for the API
    base_url = "http://oasis.caiso.com/oasisapi/SingleZip"

    # The API parameters
    params = {
        "queryname": queryname,  # the type of data you want
        "startdatetime": start,  # the start time in ISO8601 format like 20230305T07:00-0000
        "enddatetime": end,  # the end time in ISO8601 format like 20230404T07:00-0000
        "market_run_id": market,
        "node": node,
        "version": "1",  # the version of the data, 1 is the latest
        "resultformat": "6",  # retuns CSV file
    }
    try:
        # Send the GET request
        response = requests.get(base_url, params=params, timeout=1000)
        # print(response.text)
        response.raise_for_status()  # Check for request errors

        # Read the ZIP file from the response
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            # Get the name of the CSV file
            csv_file_name = zip_file.namelist()[0]
            with zip_file.open(csv_file_name) as csv_file:
                df = pd.read_csv(csv_file)
                df_total = pd.concat([df_total, df])
                monthes += 1
        print(f"Collected{monthes} monthes", datetime.now().strftime("%H:%M:%S"))

    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    # Delay to avoid too many requests error
    time.sleep(5)

# df_total = df_total.sort_values('INTERVALSTARTTIME_GMT').reset_index(drop=True)
print(df_total)

# df_total now contains all the data
print('Finished!')

Start: 20190102T00:00+0700, End: 20190105T00:00+0700
Collected{monthes} monthes 15:47:12
               <?xml version="1.0" encoding="UTF-8"?>
0   <m:OASISReport xmlns:m="http://www.caiso.com/s...
1                                   <m:MessageHeader>
2   \t<m:TimeDate>2024-01-06T23:47:07.785Z</m:Time...
3                        \t<m:Source>OASIS</m:Source>
4                  \t<m:Version>v20131201</m:Version>
5                                  </m:MessageHeader>
6                                  <m:MessagePayload>
7                                             <m:RTO>
8                              <m:name>CAISO</m:name>
9                                 <m:DISCLAIMER_ITEM>
10  <m:DISCLAIMER>The contents of these pages are ...
11                               </m:DISCLAIMER_ITEM>
12                                          <m:ERROR>
13                      <m:ERR_CODE>1000</m:ERR_CODE>
14  <m:ERR_DESC>No data returned for the specified...
15                                         </m:

# Saving df_total dataframe into CSV file

In [34]:
df_total.to_csv('df_total_raw.csv', index=False)

# Cleaning of the dataframe and saveing updated dataframe into df_total_cleaned.csv

In [4]:
df_total = pd.read_csv('df_total.csv')
df_total_cleaned = df_total[df_total['LMP_TYPE'] == 'LMP'] # removeing all rows where not LMP in the column LMP_TYPE
df_total_cleaned = df_total_cleaned.reset_index(drop=True) # reseting index
df_total_cleaned = df_total_cleaned.drop(columns=['NODE_ID_XML', 'NODE_ID', 'XML_DATA_ITEM', 'PNODE_RESMRID', 'GRP_TYPE', 'OPR_INTERVAL', 'POS', 'GROUP']) # droping column LMP_TYPE
df_total_cleaned.to_csv('df_total_cleaned.csv', index=False) # saving csv file
df_total_cleaned

/var/folders/s4/ynpwgjp142j7ls3b90kqj3_80000gn/T/ipykernel_5456/931006055.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv('df_total.csv')


,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,NODE,MARKET_RUN_ID,LMP_TYPE,MW,"<?xml version=""1.0"" encoding=""UTF-8""?>"
0,2020-06-25T17:00:00-00:00,2020-06-25T17:05:00-00:00,2020-06-25,11.0,HOLLISTR_1_N101,RTM,LMP,13.62629,NaN
1,2020-06-25T17:05:00-00:00,2020-06-25T17:10:00-00:00,2020-06-25,11.0,HOLLISTR_1_N101,RTM,LMP,14.02224,NaN
2,2020-06-25T17:10:00-00:00,2020-06-25T17:15:00-00:00,2020-06-25,11.0,HOLLISTR_1_N101,RTM,LMP,14.96814,NaN
3,2020-06-25T17:15:00-00:00,2020-06-25T17:20:00-00:00,2020-06-25,11.0,HOLLISTR_1_N101,RTM,LMP,16.06550,NaN
4,2020-06-25T17:20:00-00:00,2020-06-25T17:25:00-00:00,2020-06-25,11.0,HOLLISTR_1_N101,RTM,LMP,15.89835,NaN
...,...,...,...,...,...,...,...,...,...
307291,2023-06-27T16:35:00-00:00,2023-06-27T16:40:00-00:00,2023-06-27,10.0,HOLLISTR_1_N101,RTM,LMP,16.09520,NaN
307292,2023-06-27T16:40:00-00:00,2023-06-27T16:45:00-00:00,2023-06-27,10.0,HOLLISTR_1_N101,RTM,LMP,16.06760,NaN
307293,2023-06-27T16:45:00-00:00,2023-06-27T16:50:00-00:00,2023-06-27,10.0,HOLLISTR_1_N101,RTM,LMP,15.84760,NaN
307294,2023-06-27T16:50:00-00:00,2023-06-27T16:55:00-00:00,2023-06-27,10.0,HOLLISTR_1_N101,RTM,LMP,14.56640,NaN


In [7]:
# The base URL for the API
base_url = "http://oasis.caiso.com/oasisapi/SingleZip"

# The API parameters
params = {
    "queryname": 'AS_REQ',
    "startdate": '20221002',
    "enddate": '20221005',
    "market_run_id": 'DAM',
    "node": 'ALL',
    "version": "3",
    "resultformat": "6",
}

# Send the GET request
response = requests.get("http://oasis.caiso.com/mrtu-oasis/GroupZip?resultformat=6&groupid=DA_GROUP&startdate=20201002")
print(response.request.url)
print(response.text)

http://oasis.caiso.com/mrtu-oasis/GroupZip?resultformat=6&groupid=DA_GROUP&startdate=20201002

